In [1]:
import os
import IPython.display as ipd
import audioread
import csv
import librosa
from tqdm import tqdm_notebook
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

home = "C:\\Users\\Amar Srinivasan\\Documents\\Music-Analysis-Project"

In [2]:
genre_dict = {}

In [3]:
os.chdir(home)
with open("genre_map.csv","r") as genre_map:
    reader = csv.reader(genre_map)
    for row in reader:
        genre_dict[row[0]] = row[1]

### Constructing the Dataset

In [4]:
headers = ["Audio Mean", "Zero Crossings", "Spectral Centroid", "Rolloff"]
for i in range(1,21):
    headers.append(f"MFCC {i}")
headers.append("Genre")

In [15]:
os.chdir(home + "\\fma_small")
folders = os.listdir()
with open("song_data.csv", 'w', newline = "") as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    for f in folders:
        if f not in ['checksums', 'README']:
            os.chdir(f)
            songs = os.listdir()
            print(f)
            for song in tqdm_notebook(songs):
                try:
                    song_id = song.replace(".mp3", "").lstrip('0')
                    if song_id in genre_dict.keys():
                        song_genre = genre_dict[song_id]

                        ### FEATURE SELECTION PORTION ###
                        audio,sr = librosa.load(song)
                        audio = np.trim_zeros(audio)
                        audio_mean = audio.mean()
                        zero_crossings = sum(librosa.zero_crossings(audio, pad = False))
                        spectral_centroid = librosa.feature.spectral_centroid(audio, sr = sr).mean()              
                        mfcc = librosa.feature.mfcc(audio, sr = sr)
                        rolloff = librosa.feature.spectral_rolloff(audio, sr = sr).mean()
                        mfcc_list = [np.mean(e) for e in mfcc]

                        data_list = [audio_mean, zero_crossings, spectral_centroid, rolloff]
                        data_list += mfcc_list + [song_genre]
                        writer.writerow(data_list)
                except:
                    print(f"Error with song {song_id} in folder {f}.")
            os.chdir("..")

000



001



002



003



004


...


111



112



113



114


NotADirectoryError: [WinError 267] The directory name is invalid: 'README.txt'

<class 'str'>


In [17]:
os.getcwd()

'C:\\Users\\Amar Srinivasan\\Documents\\Music-Analysis-Project\\fma_small'

In [18]:
songs = pd.read_csv("song_data.csv")
songs.shape

(5337, 25)

In [5]:
with open("test.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["This, is, a, test"])

In [12]:
'''
import matplotlib.pyplot as plt
import librosa.display


plt.figure(figsize=(14, 5))
librosa.display.waveplot(test_song[1:15000], sr=sr)
plt.show()
'''

'\nimport matplotlib.pyplot as plt\nimport librosa.display\n\n\nplt.figure(figsize=(14, 5))\nlibrosa.display.waveplot(test_song[1:15000], sr=sr)\nplt.show()\n'

### Creating a Machine Learning Model using SVM Classifiers

In [19]:
X = songs.drop('Genre', axis = 1)
y = songs['Genre']

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
# Implementing gridsearchCV
from sklearn.model_selection import GridSearchCV

grid_param = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(), grid_param, refit = True, verbose = 2, njobs = -1)
grid.fit(X_train, y_train)

c:\users\amar srinivasan\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   1.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   2.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=   1.9s
[CV] C=0.1, gamma=1, kernel=poly .....................................


In [21]:
from sklearn.metrics import classification_report, confusion_matrix

kernels = ['Linear', 'RBF', 'Sigmoid', "Polynomial"]
for k in kernels:
    print(f"Testing SVM with {k} kernel")
    svclassifier = SVC(kernel='linear')
    svclassifier.fit(X_train, y_train)
    y_pred = svclassifier.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

[[51 19  4 42  4  6  3  7]
 [23 54 20 12  8 21  6 15]
 [ 5 16 76  1  4 16  5 13]
 [26 11  3 81  2 12  5 11]
 [ 8 13 12  2 12  9  2 11]
 [15 19  6 13  0 64 10 12]
 [30 18  6 15  4 23 13 28]
 [ 9  6  5 11  3  6  5 96]]
               precision    recall  f1-score   support

   Electronic       0.31      0.38      0.34       136
 Experimental       0.35      0.34      0.34       159
         Folk       0.58      0.56      0.57       136
      Hip-Hop       0.46      0.54      0.49       151
 Instrumental       0.32      0.17      0.23        69
International       0.41      0.46      0.43       139
          Pop       0.27      0.09      0.14       137
         Rock       0.50      0.68      0.57       141

     accuracy                           0.42      1068
    macro avg       0.40      0.40      0.39      1068
 weighted avg       0.40      0.42      0.40      1068



               precision    recall  f1-score   support

   Electronic       0.34      0.31      0.33       113
 Experimental       0.20      0.21      0.20       134
         Folk       0.47      0.42      0.45       125
      Hip-Hop       0.32      0.55      0.40       110
 Instrumental       0.15      0.14      0.14        37
International       0.39      0.49      0.44       150
          Pop       0.11      0.02      0.03       117
         Rock       0.26      0.25      0.26       106

     accuracy                           0.32       892
    macro avg       0.28      0.30      0.28       892
 weighted avg       0.30      0.32      0.30       892



,Audio,Zero Crossing Rate,Spectral Centroid,MFCC,Rolloff,Genre
2,3.134626e-04,108341,3037.939465,-0.633934,6215.998586,Hip-Hop
5,6.858170e-05,66268,2413.923644,2.423232,5434.975469,Hip-Hop
10,2.394305e-05,98403,2349.568550,2.009282,4315.856035,Pop
140,9.984850e-06,29326,1748.028062,-3.714334,3870.100836,Folk
141,9.728277e-07,40947,1655.168568,-3.491436,3317.732445,Folk
148,-7.504798e-03,59699,1707.284495,11.613784,3543.688048,Experimental
182,3.628743e-06,66618,1876.689601,7.559654,3949.201120,Rock
190,1.196952e-06,58530,1899.980137,-9.224703,3713.927542,Folk
193,4.717180e-05,51105,1583.172605,-10.606495,3205.828588,Folk
194,-1.801641e-05,48273,1546.720808,-9.238567,3086.298004,Folk


In [ ]:
# Plays a song
os.startfile(songs[0])
ipd.Audio(songs[0])

<class 'numpy.ndarray'>
<class 'numpy.int32'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'str'>
